In [2]:
import annoy
import pickle
import string
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm import tqdm_notebook 
from gensim.models import Word2Vec

In [3]:
#загрузим исходные документы
checks = pd.read_csv("check_data.csv") 
with open("Product_dict.pkl","rb") as f:
    product_dict = pickle.load(f)
checks.head()

C:\Users\miron\AppData\Local\Temp\ipykernel_40048\228411609.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  checks = pd.read_csv("check_data.csv")


,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


После небольшого анализа датасета можно выделить следующее:
- Помимо даты продажи id товара и его описания, у нас так же есть:
1. shop_id: идентефикатор магазина в котором происходила покупка
2. product_sub_category_id/product_category_id/brand_id -подкатегория/категория товара и и id его бренда, все три в целом в теории могут помочь подобрать пользователю похожий товар. Все три страдают от того, что вместо правильных значений частов встречается -1
3. quantity - колличество купленного товара, само по себе маловажно
4. contact_id - возможно является идентефикатором покупателя, точно не является идентефикатором чека, так как покупки по одному id происходят в разные дни, так же не является id кассира/кассового аппарата так как на ~850 магазинов приходится 16464366 уникальных contact_id

In [4]:
checks.drop(['name'], axis=1, inplace=True)
#Создадим признак по которому мы можем идентефицировать отдельный чек со всеми его товарами
checks['sale_id'] = checks['sale_date_date'].apply(str) + "_" + checks['contact_id'].apply(str) + "_" + checks['shop_id'].apply(str)

In [5]:
 #в попытке перевода на в дейттайм формат выдает ошибку в которой можно видеть что в sale_date_date есть не только даты, уберем не даты
checks.dropna(inplace=True)
checks = checks[checks['sale_date_date'].str.startswith("20")]
checks.reset_index(inplace=True, drop=True)
checks['sale_date_date'] = pd.to_datetime(checks['sale_date_date'])
checks.sort_values('sale_date_date', inplace=True)

In [6]:
#Поделим выборки на сплиты для валидации
#Мы рискуем разделить часть куленных одновременно предметов не делая shuffle, но на результат это повлиять сильно не должно
train, test = train_test_split(checks, test_size=0.3)

Относительно ленивое решение, лучше было сгрупировать датсет, а потом взять в разные колонки sale id и item_list(сгрупированные-id шники товаров), таким образом не было бы разделения одной покупки на две(часть в test часть в train)

In [7]:
reverse_product_dict = {j: k for k,j in product_dict.items()} # перевернем словарь для удобства поиска

In [8]:
exclude = set(string.punctuation)
morpher = MorphAnalyzer()
sw = get_stop_words("ru")

def preprocess(line):
    spls = "".join(i for i in str(line).strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [9]:
sentences = [preprocess(str(i)[str(i).find(')')+1:]) for i in reverse_product_dict.keys()] #берем слова из словаря продуктов и предобрабатываем их, убирая число в начале
sentences = [i for i in sentences if len(i) > 2]

#### Контекстные рекомендации

In [10]:
FT_model= FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

In [12]:
ft_index = annoy.AnnoyIndex(20 ,'angular')
index_dict = {}

idc = 0

for id in tqdm_notebook(product_dict.keys()):
    n_ft = 0
    index_dict[idc] = id
    vector = np.zeros(20)
    sen = product_dict[id]
    for word in preprocess(str(sen)[str(sen).find(')')+1:]):
        if word in FT_model.wv:
            vector += FT_model.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector / n_ft
    ft_index.add_item(idc, vector)
    idc += 1

# 
ft_index.build(10)

C:\Users\miron\AppData\Local\Temp\ipykernel_40048\797071751.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(product_dict.keys()):


  0%|          | 0/30418 [00:00<?, ?it/s]

True

##### Изначально был сделан общий векторизатор который складывает векторы товаров и в корзине, но его результаты оставляли желать лучшего, так что было принятно решение выдавать рекомендации по каждому товару отдельно

In [126]:
def fasttext_vec(item): # для удобства чтения сделал отдельную функцию для создания вектора для одного товара
    item = preprocess(str(item)[str(item).find(')')+1:])
    vector = np.zeros(20)
    n = 0
    for word in item:
        if word in FT_model.wv:
            vector+= FT_model.wv[word]
            n += 1
    if n>0:
        vector = vector/n
    return vector

def FT_recommend(id_list):
    dist_dict = {}
    vec_list = []
    for id in id_list:
        item_name = product_dict[id]
        if item_name not in reverse_product_dict:
            continue
        vec_list.append(fasttext_vec(item_name)) #берем вектор товара и записываем его в список
    for i in vec_list:
        rec_items = []
        rec_items = ft_index.get_nns_by_vector(i, 11) # берем 11 векторов(так как наш вектор уже в пространстве его будет выдавать первым
        vec_id = rec_items[0] #сохраняем id нашего вектора в отдельную переменную
        rec_items.pop(0) #убираем его из списка
        for j in rec_items: #проходимся по всем ближайшим веторам 
            if ft_index.get_distance(vec_id,j) != 0: 
                dist_dict[ft_index.get_distance(vec_id,j)] = j #Ддобавляем в словарь дистанцию между искходным вектором+рекомендованным и id рекомендованного
    dist_dict = sorted(dist_dict.items())[:10] #сортируем и берем 10 наиболее близких
    return [index_dict[i[1]] for i in dist_dict]

"""
def FT_recommend(id_list): 
    list_vec = np.zeros(20)
    l_len = len(id_list)
    for id in id_list:
        item_name = product_dict[id]
        if item_name not in reverse_product_dict:
            l_len -= 1
            continue
        list_vec+= fasttext_vec(item_name)
    list_vec = list_vec/l_len
    rec_items = ft_index.get_nns_by_vector(list_vec, 10+l_len)
    id_to_return = [index_dict[i] for i in rec_items]
    return [i for i in id_to_return if i not in id_list]
"""          
    
def dict_check(list):
    for i in list:
        print(product_dict[i])

In [127]:
print('Начальные товары:')
dict_check(['168308','143681','101305'])
print('Рекомендации:')
dict_check(FT_recommend(['168308','143681','101305']))

Начальные товары:
(197312) Пакет-майка 25см х 45см 906
(65326) Кординорм табл. п.п.о. 5 мг №30 738
(41123) Энап-H таб.10мг+25мг №20 738
Рекомендации:
(40894) Энап-HL таб.10мг+12,5мг №20 738
(60023) Энап-HL таб.20мг+12,5мг №20 738
(67751) Биол табл. п.п.о 5 мг №30 738
(100304) Биол табл. п.п.о 2,5 мг №30 738
(33282) Энап таб.2,5мг №20 738
(30586) Нифекард ХЛ табл. п.п.о. 30 мг. №30 738
(65312) Нипертен табл. п.п.о. 2.5 мг №30 738
(200394) Пакет-майка 40см х 70см 906
(109996) Симгал табл. п.п.о. 40 мг. №28 685
(75050) Тулип табл. п.п.о. 40 мг. №30 685


Получаем таким образом наиболие похожие товары по описанию.

### Word2Vec рекомендации по последовательностям товаров



In [17]:
#Ранее мы подготовили для групировки по корзине признак sale_id, осталось лишь сгрупировать по этому id наш датасет и скормить данные обученой w2v модели(которую еще надо обучить)
train['product_id'] = train['product_id'].apply(int).apply(str)
check_group = train.groupby('sale_id')
check_group.groups

{'2018-01-01_100232_312.0': [4743882], '2018-01-01_100608_312.0': [4725145, 4744187], '2018-01-01_1006676_332.0': [4724544, 4763456, 4740313], '2018-01-01_100886_597.0': [4739543, 4744612], '2018-01-01_101161_264.0': [4810506], '2018-01-01_1022374_43.0': [4763418, 4763502], '2018-01-01_1026344_160.0': [4740294, 4724962], '2018-01-01_1026759_160.0': [4767446, 4740359, 4767448, 4767445, 4767447], '2018-01-01_1031391_553.0': [6666931, 4826928, 6670536, 6679692], '2018-01-01_1034344_502.0': [4825663, 6679545], '2018-01-01_103741_264.0': [4712060], '2018-01-01_1038830_523.0': [4825645], '2018-01-01_103939_266.0': [4744101], '2018-01-01_1043280_332.0': [4704419, 4801655, 4690762], '2018-01-01_1043327_51.0': [4805708, 6509712], '2018-01-01_1044784_101.0': [4763505], '2018-01-01_1044919_281.0': [4763367], '2018-01-01_1045720_281.0': [4809756, 4809737, 4763451, 4704437], '2018-01-01_1045839_275.0': [4805723, 4763374], '2018-01-01_1050038_281.0': [4723618, 4763484], '2018-01-01_1051509_57.0': [4

In [18]:
# Перенесем в спиок комбинации продуктов для обучения word2vec модели, брать будем только чеки в которых 3 и более предметов
sentences = []
for group in tqdm_notebook(check_group.groups):
    products = check_group.get_group(group)['product_id'].values
    if len(products) < 3:
        continue
    sentences.append(list(products))

sentences[0]

C:\Users\miron\AppData\Local\Temp\ipykernel_40048\70507501.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for group in tqdm_notebook(check_group.groups):


  0%|          | 0/5457442 [00:00<?, ?it/s]

['62087', '120171', '69028']

In [20]:
W2V_model = Word2Vec(sentences, vector_size=10)

In [27]:
def W2V_prediction(item_list):
    vec = np.zeros(10)
    len_list= len(item_list)
    for item in item_list:
        if item not in product_dict:
            len_list -=1
            continue
        vec += W2V_model.wv[item]
    return [i[0] for i in W2V_model.wv.similar_by_vector(vec / len_list, 10)]

In [30]:
print('Начальные товары:')
dict_check(['168308','143681','101305'])
print('Рекомендации:')
dict_check(W2V_prediction(['168308','143681','101305']))

Начальные товары:
(197312) Пакет-майка 25см х 45см 906
(65326) Кординорм табл. п.п.о. 5 мг №30 738
(41123) Энап-H таб.10мг+25мг №20 738
Рекомендации:
(38101) Капотен тб 25мг N56 738
(43433) Капотен тб 25мг N40 738
(40897) Энап-HL таб.10мг+12,5мг №20 738
(78752) Атенолол Никомед табл. п.п.о. 50 мг. №30 738
(38772) Коринфар таб. п.о. 10мг фл №50 738
(110948) Энап таб.20мг №60 738
(15924) Кордипин ретард табл. п.о. ретард 20 мг. №30 -1
(39158) Коринфар таб. п.о. 10мг фл №100 738
(64563) Эналаприл Гексал таб.10мг №20 738
(45741) Капозид тб 50мг N28 738


В отличии от контекстных рекомендаций, вывод w2v по последовательностям не так очевиден, по крайней мере для не фармацевтов. Что мы можем сделать для его проверки это взять заранее отложенный тестовый датасет, сгруппировать его так же как сгруппировали тренировочный, взять в нем чеки из трех и более товаров и перенести один или несколько товаров в отдельную колонку для того чтобы посмотреть порекомендует ли нам этот товар наша модель. После чего провести тестовую выборку по нашей функции и посчитать точность например через Precision@N

### Двухстадийная рекомендательная система 
#### Выбор кандидатов
Возьмем 7 w2v и 3 FT, обусловлено это тем, что качество, как минимум на первый взгляд, у w2v модели выше, FT уделяет слишком большое внимание граммовкам и остальному дополнительному тексту по типу "аб.10мг+12,5мг №20 738". Cсоотношени может изменится при более глубоком тестировании, но для начала возьмем так.е
#### Ранжирование
Существует огромное множестов подходов ранжирования. Учитывая что мы ищем рекомендации по товарам в корзине и тому что ранжируем мы две модели, модель которая ищет наиболее близкий товар по названию, и модель которая предсказывает товар по группам товаров в корзине.(своебразные версии "похожие товары" и "Часто покупаемые вместе")В голову приходятя два варианта ранжирования.
1. Более сложное - берем все записанные группы товаров в датаесете, вынимает случайный товар в отдельный столбец и обучаем на этом модель. Которая впоследствии будет предсказывать "уверенность" в наших ответа, по ней и ранжируемх.
2. Менее сложное - в качестве системы скоринга берем схожесть из w2v и 1 - дистанция*2(в качестве доп. веса) между вещью в корзине и предсказанной вещью из FT, считаем это за оценки и ранжируем по ним.


Пойдем по пути наименьшего сопротивление и выберем второй вариант

Оставим предыдущие функции не тронутыми и переделаем их под наши нужды отдельно.

In [227]:
def FT_for_range(id_list):
    dist_dict = {}
    vec_list = []
    for id in id_list:
        item_name = product_dict[id]
        if item_name not in reverse_product_dict:
            continue
        vec_list.append(fasttext_vec(item_name)) #берем вектор товара и записываем его в список
    for i in vec_list:
        rec_items = []
        rec_items = ft_index.get_nns_by_vector(i, 11) # берем 11 векторов(так как наш вектор уже в пространстве его будет выдавать первым
        vec_id = rec_items[0] #сохраняем id нашего вектора в отдельную переменную
        rec_items.pop(0) #убираем его из списка
        for j in rec_items: #проходимся по всем ближайшим веторам 
            if ft_index.get_distance(vec_id,j) != 0: 
                dist_dict[ft_index.get_distance(vec_id,j)] = j #Ддобавляем в словарь дистанцию между искходным вектором+рекомендованным и id рекомендованного
    dist_dict = dict(sorted(dist_dict.items())[:3]) #сортируем и берем 3 наиболее близких
    return dist_dict

def W2V_for_range(item_list):
    vec = np.zeros(10)
    len_list= len(item_list)
    for item in item_list:
        if item not in product_dict:
            len_list -=1
            continue
        vec += W2V_model.wv[item]
    return W2V_model.wv.similar_by_vector(vec / len_list, 7)

def ranging(item_list):
    combined_dict = dict(W2V_for_range(item_list)) #берем 7 наиболее вероятных предсказаний из w2v
    FT_dict = FT_for_range(item_list) #берем 3 из FT
    FT_dict = {index_dict[j]: 1-k*2 for k,j in FT_dict.items()} # погодняем ft под w2v меняем индекс на id товара, и "оцениваем" расстояние вектора
    combined_dict.update(FT_dict)
    combined_dict = {j: k for k,j in combined_dict.items()}
    return [i[1] for i in sorted(combined_dict.items(), reverse=True)]

In [228]:
print('Начальные товары:')
dict_check(['168308','143681','101305'])
print('Рекомендации:')
dict_check(ranging(['168308','143681','101305']))

Начальные товары:
(197312) Пакет-майка 25см х 45см 906
(65326) Кординорм табл. п.п.о. 5 мг №30 738
(41123) Энап-H таб.10мг+25мг №20 738
Рекомендации:
(40894) Энап-HL таб.10мг+12,5мг №20 738
(60023) Энап-HL таб.20мг+12,5мг №20 738
(67751) Биол табл. п.п.о 5 мг №30 738
(38101) Капотен тб 25мг N56 738
(43433) Капотен тб 25мг N40 738
(40897) Энап-HL таб.10мг+12,5мг №20 738
(78752) Атенолол Никомед табл. п.п.о. 50 мг. №30 738
(38772) Коринфар таб. п.о. 10мг фл №50 738
(110948) Энап таб.20мг №60 738
(15924) Кордипин ретард табл. п.о. ретард 20 мг. №30 -1


In [229]:
ranging(['168308','143681','101305'])

['62144',
 '126081',
 '124362',
 '102814',
 '115626',
 '142899',
 '150856',
 '124198',
 '93397',
 '59086']